In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import math

In [61]:
sr = pd.read_csv("/Users/Jeanne/Documents/personal/Auction Results/soldresults10yr.csv")
ped = pd.read_csv("/Users/Jeanne/Documents/personal/Auction Results/pedigree.csv")
proglist = pd.read_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\proglist.csv')
prograces = pd.read_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\428prog.csv')
raceresults = pd.read_csv("/Users/Jeanne/Documents/personal/Auction Results/hrn.csv")

In [52]:
goldencentsAuction = sr[sr['Siref'] == 'Goldencents']
goldencentsPedigree = ped[ped['Name'] == 'Goldencents']
goldencentsProgeny = proglist[proglist['Name'] == 'Goldencents']
goldencentsRaceResults = raceresults[raceresults['Horse'] == 'Goldencents']

In [50]:
goldencentsAuction.loc[:,['Damf','Siref','Pricef','OBSDate']].head()

,Damf,Siref,Pricef,OBSDate
2526,Citizen Emma,Goldencents,25000.0,2018-01-01
2767,Gimme Sugar,Goldencents,21000.0,2018-01-01
2939,Little Maryruth,Goldencents,25000.0,2020-01-01
3019,Our Rockin Ruby,Goldencents,19000.0,2020-01-01
3027,Past Due,Goldencents,18000.0,2020-01-01


In [25]:
goldencentsPedigree

,Unnamed: 0,Name,S,SS,SSS,SSD,SD,SDS,SDD,D,DS,DSS,DSD,DD,DDS,DDD
16133,16133,Goldencents,Into Mischief,Harlan's Holiday,Harlan,Christmas in Aiken,Leslie's Lady,Tricky Creek,Crystal Lady,Golden Works,Banker's Gold,Forty Niner,Banker's Lady,Body Works,Bold Ruckus,Kinto


KEY: S sire, SS sire's sire, SSS sire's sire's sire, SSD sire's sire's dam, DS dam's sire, DSD dam's sire's dam etc...

In [33]:
goldencentsProgeny

,Unnamed: 0,Name,Progeny_raw
5404,18813,Goldencents,Page 1 of 3\nBy My Standards (USA)\nMr. Money ...


I will make this data more palatable, ignore the code below. It will show the top 5 progeny (roughly) of Goldencents:

In [36]:
temp = goldencentsProgeny['Progeny_raw'].tolist()

gcprogeny = list()

parser = temp[0].split('\n')
for p in parser:
    gcprogeny.append(p.replace(r' (USA)',''))

gcprogeny = [x for x in gcprogeny if not 'Page 1 of' in x]
gcprogeny[0:5]

['By My Standards',
 'Mr. Money',
 'Wildman Jack',
 'Going To Vegas',
 'Phantom Currency']

Check if By My Standards shows Goldencents as sire in pedigree:

In [40]:
ped[ped['Name'] == 'By_My_Standards']

,Unnamed: 0,Name,S,SS,SSS,SSD,SD,SDS,SDD,D,DS,DSS,DSD,DD,DDS,DDD
16745,16745,By_My_Standards,Goldencents,Into Mischief,Harlan's Holiday,Leslie's Lady,Golden Works,Banker's Gold,Body Works,A Jealous Woman,Muqtarib,Gone West,Shicklah,Miss Free Bird,Fly So Free,Stormfeather


Is By My Standards in the auction results?

In [57]:
goldencentsAuction[goldencentsAuction['Damf'] == 'A Jealous Woman'].loc[:,['Damf','Siref','Pricef','OBSDate']]

,Damf,Siref,Pricef,OBSDate
23146,A Jealous Woman,Goldencents,150000.0,2018-04-01


See Goldencents race results:

In [55]:
goldencentsRaceResults.tail()

,Unnamed: 0,Date,Horse,Fin,Tr,Dist.,Sf.,Race,Cnd,1st,2nd,3rd,Time
1577,1577,03/09/13-R7,Goldencents,4,SA,1 1/16 m,D,San Felipe-G2,3,Hear The Ghost,Flashback,Tiz a Minister,1:42.34
1578,1578,01/05/13-R3,Goldencents,1,SA,1 mile,D,Sham-G3,3,Goldencents,Den's Legacy,Manando,1:36.50
1579,1579,11/17/12-R7,Goldencents,1,DeD,1 1/16 m,D,Delta Downs Jackpot-G3,2,Goldencents,Bern Identity,Mylute,1:44.89
1580,1580,10/06/12-R8,Goldencents,2,Bel,1 mile,D,Champagne-G1,2,Shanghai Bobby,Goldencents,Fortify,1:35.55
1581,1581,09/02/12-R6,Goldencents,1,Dmr,5 1/2 f,S,MSW,2,Goldencents,Wild Dude,Storm Power,1:02.79


See By My Standards Race Results

In [54]:
raceresults[raceresults['Horse'] == 'By My Standards'].tail()

,Unnamed: 0,Date,Horse,Fin,Tr,Dist.,Sf.,Race,Cnd,1st,2nd,3rd,Time
30001,10,03/23/19-R13,By My Standards,1st,FG,1 1/8 m,D,Louisiana Derby-G2,3,By My Standards,Spinoff,Sueno,1:49.53
30002,11,02/16/19-R7,By My Standards,1st,FG,1 1/16 m,D,MSW,3,By My Standards,Awesome Wok N Roll,Tapit Wise,1:45.04
30003,12,01/19/19-R7,By My Standards,3rd,FG,1 70y,D,MSW,3,Frank'sgunisloaded,Comedian,By My Standards,1:44.87
30004,13,12/22/18-R4,By My Standards,2nd,FG,1 1/16 m,D,MSW,2,Gun It,By My Standards,Courtyard,1:46.13
30005,14,11/24/18-R4,By My Standards,2nd,CD,6 1/2 f,D,MSW,2,Jersey Agenda,By My Standards,Malpais,1:17.58


See A Jealous Woman's Race Results

In [62]:
raceresults[raceresults['Horse'] == 'A Jealous Woman'].tail()

,Unnamed: 0,Date,Horse,Fin,Tr,Dist.,Sf.,Race,Cnd,1st,2nd,3rd,Time
30026,20,02/19/10-R7,A Jealous Woman,1st,SA,1 1/16 m,T,AlwOC,4,A Jealous Woman,Welken,Michael's Rose,1:46.77
30027,21,01/10/10-R5,A Jealous Woman,1st,SA,1 mile,T,Allow,4+,A Jealous Woman,Devine Actress,Playa Bonita,1:34.52
30028,22,12/20/09-R5,A Jealous Woman,1st,Hol,6 f,T,AlwOC,3+,A Jealous Woman,Quid Non,Princess Suances,1:08.83
30029,23,11/22/09-R9,A Jealous Woman,1st,Hol,6 f,D,MSW,3+,A Jealous Woman,Susan's Wildcat,Krismas Prospect,1:09.81
30030,24,07/19/09-R7,A Jealous Woman,2nd,Hol,6 f,D,MSW,3+,Speedin Excess,A Jealous Woman,Streakin Excess,1:10.29


See A Jealous Woman's Progeny

In [64]:
proglist[proglist['Name'] == 'A Jealous Woman']

,Unnamed: 0,Name,Progeny_raw
17300,30709,A Jealous Woman,By My Standards (USA)\nGentle Soul (USA)\nRede...


In [66]:
JWprogeny = proglist[proglist['Name'] == 'A Jealous Woman']
temp = JWprogeny['Progeny_raw'].tolist()

jwprogeny = list()

parser = temp[0].split('\n')
for p in parser:
    jwprogeny.append(p.replace(r' (USA)',''))

jwprogeny = [x for x in jwprogeny if not 'Page 1 of' in x]
jwprogeny[0:5]

['By My Standards', 'Gentle Soul', 'Redemption Time', '   ']

Interestingly, Gentle Soul and Redemption Time do not really appear to have raced. So the trail stops here for A Jealous Woman. By My Standards was a foal sold for 130,000 from Goldencents and A Jealous Woman in April 2018. Racing results for By My Standards, Goldencents, and A Jealous Woman can all be pulled from one dataset. Lists of all progeny and pedigree for both Goldencents and A Jealous Woman can also be collected.